CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import numpy as np
import random
import pandas as pd

#!pip install pyomo
#import pyomo.environ as pyEnv
#!pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import copy
import time

#list_n=[100,500,1000,2000,5000]
#list_m=[400,2000,4000,8000,20000]
#infected_percentage_list = [0.15,0.1,0.05]
#risky_percentage_list = [0.8,0.5,0.3]

adjacency_df=pd.DataFrame(pd.read_csv(r"C:\\Users\\utku\Desktop\\bitirme\\\data\\adjaceny_df\\adjacency_df-n100-m400-run1-risky40-inf10_utku.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2

#sets
setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()


In [2]:
total_weight_list=(adjacency_df.sum(axis=0)[:-3]).copy()
maximum_weight = round(max(total_weight_list)+0.5)
adjacency_df.iloc[:,:-2]=adjacency_df.iloc[:,:-2].div(maximum_weight,axis=0)
error_constant = min(adjacency_df["Threshold"])/1000

In [3]:
maximum_weight

9

In [4]:
min(adjacency_df["Threshold"])

0.0015555555555555555

In [5]:
adjacency_df.iloc[23]

1             0.000000
2             0.000000
3             0.000000
4             0.000000
5             0.000000
                ...   
99            0.000000
100           0.000000
Threshold     0.001556
IsInfected    0.000000
IsRisky       0.000000
Name: 24, Length: 103, dtype: float64

total_weight=0
adjacency_df["Total Impact"] = np.zeros((n,1))
for node in adjacency_df.index:
    total_weight=0
    adjacency_df.at[node,"Total Impact"] = adjacency_df.sum(axis=1)[node]
    for neighbor in adjacency_df.columns[:-4]:
        adjacency_df.at[node,neighbor]=adjacency_df.at[node,neighbor]/total_weight
    adjacency_df.at[node,"Threshold"]=adjacency_df.at[node,"Threshold"]/total_weight


In [6]:
start_time = time.time()

#parameters
wij = [[adjacency_df.loc[i,j] for i in setV] for j in setV]
ti = [adjacency_df.loc[i,"Threshold"] for i in setV]
#Xi = [adjacency_df.loc[i,"IsRisky"] for i in setV]
Xi=[]
for i in setV:
    check_zero = adjacency_df.loc[i,"IsRisky"]
    if(check_zero==0):
        check_zero+=1/(10000*n+1)
    Xi.append(check_zero)

#Blocking_Capacity = int((n) / 10) 
Blocking_Capacity = int((n) / 10)*2

#model
Math_Model = gp.Model("Math_Model")

#variables
Yi = Math_Model.addVars(setV,vtype=GRB.BINARY,name="Yi") #Bunun yerine SetS'de tanımlı olabilir!!!!
Ii = Math_Model.addVars(setV,vtype=GRB.BINARY, obj=Xi, name="Ii")
Multi= Math_Model.addVars(setV,vtype=GRB.BINARY,name="Multi") #(1-Y[i])*(1-I[i])

#constraints
# Y[i] +I[i] <= 1   (1)

# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))


for i in setS:
    Math_Model.addConstr(lhs=(Ii[i]+Yi[i]), sense=GRB.GREATER_EQUAL, rhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV) - ti[i-1] + error_constant,name="LTM_Model".format(i))



Math_Model.addConstr(lhs=gp.quicksum(Yi[i] for i in setV) , sense=GRB.LESS_EQUAL, rhs=Blocking_Capacity,name="Bloklama kapasitesinden fazla bloklama yapılamaz.".format(i))

for i in setP:
    Math_Model.addConstr(Ii[i], sense=GRB.EQUAL, rhs=1, name="Başlangıçta enfekte olanların I[i] parametresi bire eşittir.".format(i))


   
Math_Model.write("out.lp")
#Math_Model.computeIIS()
#Math_Model.write("model.ilp")

Math_Model.ModelSense = GRB.MINIMIZE
Math_Model.optimize()

#for v in Math_Model.getVars():
    #if(v.x>= -0.0001):
        #print('%s %g' % (v.varName, v.x))

vars = Math_Model.getVars()

adjacency_df['Blocked']=np.zeros((n,1))

adjacency_df['Ending_Infected']=np.zeros((n,1))


for i in range(1,n+1):
    adjacency_df.at[i,'Blocked']=vars[i-1].x
    adjacency_df.at[i,'Ending_Infected']=vars[i+n-1].x

Objective_Value = Math_Model.ObjVal
Objective_Value = round(Objective_Value-0.5)
end_time = time.time()

adjacency_df.loc[(adjacency_df.loc[:,'IsRisky'] == 1) & (adjacency_df.loc[:,'Ending_Infected'] == 1) ,['Threshold','IsInfected','IsRisky','Blocked','Ending_Infected']]

adjacency_df['Blocked'].to_csv("Math_Model_NODE-n"+str(n)+'-m'+str(m)+'-OBJ'+str(Objective_Value)+".csv",index=False)       

Academic license - for non-commercial use only - expires 2021-06-09
Using license file C:\Users\utku\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 201 rows, 300 columns and 1223 nonzeros
Model fingerprint: 0x63dce50c
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 2e+01]
Found heuristic solution: objective 40.0000600
Presolve removed 66 rows and 143 columns
Presolve time: 0.01s
Presolved: 135 rows, 157 columns, 843 nonzeros
Variable types: 0 continuous, 157 integer (157 binary)

Root relaxation: objective 1.358850e-05, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0

In [7]:
print(end_time-start_time)

4.995493173599243


In [8]:
adjacency_df

,1,2,3,4,5,6,7,8,9,10,...,96,97,98,99,100,Threshold,IsInfected,IsRisky,Blocked,Ending_Infected
1,0.000000,0.0,0.0,0.0,0.028556,0.0,0.0,0.0,0.0,0.0,...,0.015333,0.0,0.0,0.0,0.0,0.081444,0.0,1.0,-0.0,1.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.011889,0.0,0.0,1.0,-0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.028222,0.0,1.0,0.0,1.0
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.023556,0.0,1.0,-0.0,-0.0
5,0.028556,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.072667,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.015333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.006444,0.0,1.0,0.0,1.0
97,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.106333,0.0,0.0,-0.0,-0.0
98,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.105111,0.0,1.0,-0.0,-0.0
99,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.052111,1.0,0.0,0.0,1.0


In [9]:
sayi=7.0001-0.5
round(sayi)

7

In [10]:
Objective_Value

13

In [11]:
adjacency_df

,1,2,3,4,5,6,7,8,9,10,...,96,97,98,99,100,Threshold,IsInfected,IsRisky,Blocked,Ending_Infected
1,0.000000,0.0,0.0,0.0,0.028556,0.0,0.0,0.0,0.0,0.0,...,0.015333,0.0,0.0,0.0,0.0,0.081444,0.0,1.0,-0.0,1.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.011889,0.0,0.0,1.0,-0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.028222,0.0,1.0,0.0,1.0
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.023556,0.0,1.0,-0.0,-0.0
5,0.028556,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.072667,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.015333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.006444,0.0,1.0,0.0,1.0
97,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.106333,0.0,0.0,-0.0,-0.0
98,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.105111,0.0,1.0,-0.0,-0.0
99,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.052111,1.0,0.0,0.0,1.0


In [12]:
ati=adjacency_df['Blocked'].tolist()

In [13]:
adjacency_df.loc[adjacency_df['Ending_Infected']>0.01,'IsRisky'].sum()

13.0

In [14]:
sum(ati)

20.0